<a href="https://colab.research.google.com/github/masatsuneki/hello-world/blob/master/20200921_Masa_KTP_DCGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

このノートブックは、下記の記事をベースにDCGANを利用した画像の生成を行うサンプルとなっています。

https://qiita.com/enmaru/items/01c3abccd4b132865686

#前書き
DCGAN(Deep Convolutional GAN)は2015年11月に論文が公開された、CNN(畳み込みニューラルネットワーク)を利用した、教師なし学習による画像の生成です。

元論文URL : https://arxiv.org/abs/1511.06434

それまで、CNNを用いた実例は教師あり学習がメインで教師なし学習は注目されていませんでした。

それは、CNNを使いGANを構成すると、中々生成がうまくいかなかったためです。しかし、この論文ではその学習を安定させるための方法がまとめられています。

本ノートブックは、その方法をTensorFlowで実装したものを、極力簡単に試せるようになっています。

(現在では、さらに精度の高い画像生成を行うGANも公開されています。)

#コード実行前に行う前に ※必須
## colabの設定
DCGANの学習を高速で行うには、GPU設定が必要です。

そのため、上部メニューより下記の設定を行ってください。

**ランタイム > ランタイムのタイプを変更 > 「ハードウェアアクセラレータ」を「None」から「GPU」に変更 > 保存**

この設定は、インスタンスが新しくなるたび(12時間で強制的にリセット)毎回必要になります。

##データセットの準備

学習に使う画像は、下記の要件に沿って準備してGoogleドライブにアップしておいてください。「colab」などディレクトリを切ってアップしておいた方が管理しやすいかと思います。

*   png形式
*   exif情報の削除
*   画像の大きさの統一 (デフォルトでは縦横108px)
*   画像枚数は64枚～
*   zipに圧縮(フォルダごと圧縮でなく、画像をすべて選択の上圧縮してください。サブディレクトリが発生しないように)



#実行の流れ
下記に記載されているコマンドを一行ずつ実行していけば、実行を進められるようになっています。

実行は、各セルにカーソルを合わせると出てくる実行ボタンをクリックするか、セル内でフォーカスがあっている状態で「Shift+Enter」を入力してください。

また、Googleドライブ上の画像のデータセットを選ぶ部分では、適宜自身で用意したzipファイルを設置した場所にパスを書き換えてください。

In [ ]:
# ホームディレクトリに移動
cd /home

In [ ]:
# githubからDCGANのコードをコピー
!git clone https://github.com/carpedm20/DCGAN-tensorflow

下記コマンドを実行すると、Googleドライブに接続するための確認ウインドウが出てきますので、案内に沿って設定を完了してください。

In [ ]:
# Googleドライブに接続
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# データセットを保存するディレクトリの作成・移動
!mkdir DCGAN-tensorflow/data

In [ ]:
cd DCGAN-tensorflow/data

下記コマンドで、Googleドライブに保存されたデータセットをColabにコピーします。

このままだと、Googleドライブ上の「colab」フォルダの「gan-imgset.zip」というファイルをコピーする状態になっているため、適宜自身でファイルを保存したパスに書き換えた上で実行してください。

以降も、ファイル名が書かれた部分はそれぞれ書き換えて実行してください。

In [ ]:
!cp '/content/drive/My Drive/colab/gan-imgset.zip' '/home/DCGAN-tensorflow/data'

In [ ]:
!unzip -q gan-imgset.zip -d gan-imgset

In [ ]:
# DCGANディレクトリに移動
cd /home/DCGAN-tensorflow/

下記コマンド実行後、学習が開始されます。

##各種オプション
###入力画像のサイズ指定

--input_height=100

(下記実行例では、画像の縦幅が100pxのものを学習する設定になっています。108pxで準備している場合は、このオプションを削除するか--input_height=108と指定して実行してください。)

### 学習画像の中心から切り抜く
--crop

### 画像のファイル名指定
--input_fname_pattern="*_cropped.png"

(上記であれば、ファイル名末が「_cropped.png」になっている画像だけを学習に使用)

### 前回の学習結果を利用する
--checkpoint_dir = './training_checkpoints'

### 生成画像を出力する頻度
--sample_freq=500
(epochを大きくする場合は、この値も大きくして良いかも。デフォルト : 200)

### 何回繰り返して学習させるか
--epoch=25

学習回数を増やすほど、生成の精度は基本的に向上していきます。

学習時間は、データセットの画像枚数に比例しますが、数千枚程度であれば、epoch25で10分ほどで完了します。

epochを数千回にすると、ほどよく学習が収束していきますが、当然学習時間も延びていきます。

学習中、ブラウザが多くのメモリを使用するため、他の作業を行うことが困難になるため注意してください。

その他オプションは、適宜元コードを参照してください。

https://github.com/carpedm20/DCGAN-tensorflow/blob/master/main.py

In [ ]:
# DCGANの実行
!python2 main.py --dataset gan-imgset --input_height=100 --input_fname_pattern="*.png" --epoch=25 --sample_freq=100 --train --crop

## 学習結果の表示・保存

*   画像の表示
*   画像単体の保存
*   画像のGIFアニメ化/ダウンロード
*   各結果をドライブにコピー

In [ ]:
# 学習結果が保存されたディレクトリに移動
cd /home/DCGAN-tensorflow/out

In [ ]:
# ファイル一覧の表示
ls

表示されたディレクトリ名の中で最新のものをコピーして、下記コマンドの「***」の部分を書き換え、実行してください。

In [ ]:
cd ***

### 画像の表示

In [ ]:
# ファイル一覧の表示
ls samples

表示したい画像ファイル名を選び、下記コマンド中の「train_***.png」の部分をその名前に書き換えて実行してください。

学習する画像の枚数が少なく、epoch25回など学習回数が少ない場合、画像が生成されません。その場合、epoch100など回数を増やした上で改めて学習を実行してください。

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

#画像内に線を表示しない
fig,ax = plt.subplots()
ax.tick_params(labelbottom="off",bottom="off")
ax.tick_params(labelleft="off",left="off")
ax.set_xticklabels([]) 
ax.axis('off')
#画像の読み込み
im = Image.open("samples/train_***.png")
#画像をarrayに変換
im_list = np.asarray(im)
#貼り付け
plt.imshow(im_list)
#表示
plt.show()

### 画像単体のダウンロード

In [ ]:
from google.colab import files
files.download('samples/train_***.png')

### 画像のGIFアニメ化/ダウンロード

In [ ]:
from PIL import Image
import glob

files = sorted(glob.glob('./samples/*.png'))
images = list(map(lambda file: Image.open(file), files))

images[0].save('./samples/out.gif', save_all=True, append_images=images[1:], duration=300, loop=0)

In [ ]:
from google.colab import files
files.download('./samples/out.gif')

### 各結果をGoogleドライブにコピー

In [ ]:
# 生成画像すべてをGoogleドライブにコピー
# 保存先は適宜書き換えて実行してください。
!mkdir "/content/drive/My Drive/colab/gan"

In [ ]:
!cp -r samples "/content/drive/My Drive/colab/gan/samples1"

In [ ]:
# チェックポイントをGoogleドライブにコピー
# 保存先は適宜書き換えて実行してください。
#
# colabのインスタンスがリセットされても、学習のチェックポイントを、
# Google Driveからコピーしてきて実行可能になります。 
!cp -r checkpoint "/content/drive/My Drive/colab/gan/checkpoint1"